In [1]:
import os
os.chdir('../')
print(os.getcwd())

/DATA/rahul_1911mt11/multi-tasking/EEG-Multi-Tasking


In [2]:
import torch 
import random
import traceback
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [3]:
gpu = 'cuda:3'

In [4]:
# Device configuration
device = torch.device(gpu if torch.cuda.is_available() else 'cpu')

# Hyper parameters
episodes = 100
num_epochs = 1
num_classes = 2
batch_size = 25
lr1 = 0.0000001
lr2 = 0.000001

In [5]:
from src.dataset import single_task_dataset
from src.dataset import multi_task_dataset

SyntaxError: invalid syntax (dataset.py, line 26)

In [ ]:
df = pd.read_csv('input/data.csv')

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df.shape

In [ ]:
def conv_block(in_channels, out_channels):
    '''Convolution Block of 3x3 kernels + batch norm + maxpool of 2x2'''

    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 5, padding=2),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

In [ ]:
class TaskNet(nn.Module):
    def __init__(self):
        super(TaskNet, self).__init__()

        self.conv1 = conv_block(1, 20)
        self.conv2 = conv_block(20, 40)
        self.conv3 = conv_block(40, 60)

        self.head = nn.Linear(9 * 4 * 60, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        # Features of shape (batch_size, 64)
        feat = x.reshape(x.size(0), -1)

        # Output
        out = self.head(feat)

        return out

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

#         self.conv1 = conv_block(1, 20)
#         self.conv2 = conv_block(20, 40)
#         self.conv3 = conv_block(40, 60)
        self.head1 = nn.Linear(9 * 4 * 60 , 100)
        self.head2 = nn.Linear(100, 50)
        self.head3 = nn.Linear(50, 20)
        self.head4 = nn.Linear(20, 2)
        
        task_model = TaskNet().to(device)
        try:
            checkpoint = torch.load('models/adversarial_train_real/tasknet.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
            task_model.load_state_dict(checkpoint['weights'])
#             print("--------- Successfully loaded previous model weights --------------------")
        except:
            print('got error in ConvNet class')
#             traceback.print_exc()
            print("-------Creating task net from scratch -----------")
        self.conv11 = task_model.conv1
        self.conv22 = task_model.conv2
        self.conv33 = task_model.conv3


    def forward(self, x):
        
#         x1 = self.conv1(x)
#         x1 = self.conv2(x1)
#         x1 = self.conv3(x1)
        
        x2 = self.conv11(x)
        x2 = self.conv22(x2)
        x2 = self.conv33(x2)
        
        

        # Features of shape (batch_size, 64)
#         feat1 = x1.reshape(x1.size(0), -1)
        feat2 = x2.reshape(x2.size(0), -1)
#         feat  = torch.cat((feat1, feat2),1)
  
        # Output
        out = self.head1(feat2)
        out = F.relu(out)
        out = self.head2(out)
        out = F.relu(out)
        out = self.head3(out)
        out = F.relu(out)
        out = self.head4(out)

        return out

In [ ]:
# Train the model

for episode in range(episodes):
    
    n = random.randint(1,9)

    df_train = df.loc[(df.Type=='train') & (df.Subject!= 10)].reset_index(drop = True)
    train_dataset = single_task_dataset(df_train)
    train_dataset = multi_task_dataset(df_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size, 
                                               shuffle=True)
    
    model1 = ConvNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_real/convnet.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
        model1.load_state_dict(checkpoint['weights'])
        print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("error while loading convnet weights")
        traceback.print_exc()
        print("-------Creating convnet from scratch -----------")
    
    model2 = TaskNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_real/tasknet.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
        model2.load_state_dict(checkpoint['weights'])
        print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("error while loading tasknet weights")
        traceback.print_exc()
        print("-------Creating task net from scratch -----------")
        

    # Loss and optimizer
    criterion1 = nn.CrossEntropyLoss()
    optimizer1 = torch.optim.Adam(model1.parameters(), lr=lr1)

    criterion2 = nn.CrossEntropyLoss()
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=lr2)
    


    total_step = len(train_loader)
    for epoch in range(num_epochs):


        for i, (images, labels, subjects) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            subjects = subjects.to(device)

            # Forward pass 1
            outputs1 = model1(images)
            loss1 = criterion1(outputs1, labels)
#             import pdb;
#             pdb.set_trace()



            # Forward pass 2
            outputs2 = model2(images)
            loss2 = criterion2(outputs2, subjects)

            # Backward and optimize 1
            optimizer1.zero_grad()
            (loss1).backward()
            optimizer1.step()

            # Backward and optimize 2
            optimizer2.zero_grad()
            (loss2 + loss1.item()).backward()
            optimizer2.step()

            torch.save({'weights': model2.state_dict()},'models/adversarial_train_real/tasknet.pth')
            torch.save({'weights': model1.state_dict()},'models/adversarial_train_real/convnet.pth')

            if (i+1) % 10000 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss1.item()))
    print(f"-------------  EPISODE : {episode}  ------------")
    print("------------------------------------------------")
    train_accuracy_model1()
    test_accuracy_model1()
    train_accuracy_model2()
    test_accuracy_model2()
    print("------------------------------------------------", '\n')

In [ ]:
# Test the model

def test_accuracy_model1():
    
    df_test = df.loc[(df.Type=='test') & (df.Subject!=10)].reset_index(drop = True)
    test_dataset = multi_task_dataset(df_test)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    
    model1 = ConvNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_real/convnet.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
        model1.load_state_dict(checkpoint['weights'])
    #         print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("-------Creating convnet from scratch -----------")
    

    model1.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels, subjects in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model1(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test Accuracy: {} %'.format(100 * correct / total))


# Test the model

def train_accuracy_model1():
    
    df_train = df.loc[(df.Type=='train') & (df.Subject!=10)].reset_index(drop = True)
    train_dataset = multi_task_dataset(df_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    
    model1 = ConvNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_real/convnet.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
        model1.load_state_dict(checkpoint['weights'])
    #         print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("-------Creating convnet from scratch -----------")

    model1.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels, subjects in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model1(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Train Accuracy: {} %'.format(100 * correct / total))

In [ ]:
# Test the model
def test_accuracy_model2():
    
    df_test = df.loc[(df.Type=='test') & (df.Subject!=10)].reset_index(drop = True)
    test_dataset = multi_task_dataset(df_test)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    
    model2 = TaskNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_real/tasknet.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
        model2.load_state_dict(checkpoint['weights'])
    #         print("--------- Successfully loaded previous model weights --------------------")
    except:
        print("-------Creating task net from scratch -----------")
    
    
    model2.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels, subjects in test_loader:
            images = images.to(device)
            labels = subjects.to(device)
            outputs = model2(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
#             print(predicted.item(), labels.item())

    print('Test Accuracy for tasknet: {} %'.format(100 * correct / total))

# Test the model
def train_accuracy_model2():
    
    df_train = df.loc[(df.Type=='train') & (df.Subject!=10)].reset_index(drop = True)
    train_dataset = multi_task_dataset(df_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1, 
                                               shuffle=True)
    
    model2 = TaskNet().to(device)
    try:
        checkpoint = torch.load('models/adversarial_train_real/tasknet.pth', map_location=torch.device(gpu if torch.cuda.is_available() else 'cpu'))
        model2.load_state_dict(checkpoint['weights'])
    #         print("--------- Successfully loaded previous model weights --------------------")
    except:
        traceback.print_exc()
        print("-------Creating task net from scratch -----------")
    
    
    model2.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels, subjects in train_loader:
            images = images.to(device)
            labels = subjects.to(device)
            outputs = model2(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
#             print(predicted.item(), labels.item())

    print('Train Accuracy for tasknet: {} %'.format(100 * correct / total))

In [ ]:
train_accuracy_model1()
test_accuracy_model1()
train_accuracy_model2()
test_accuracy_model2()

In [ ]:
train_accuracy_model2()

In [ ]:
test_accuracy_model2()

In [8]:
from PIL import Image
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [12]:
class single_task_dataset(Dataset):
    def __init__(self,df, transform=None):
        """
        Args:
            df (pandas dataframe): dataframe with path and labels of all EEG-images
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self,index):
        # import pdb; pdb.set_trace()
        label = self.df.Label.loc[index]
        label = torch.tensor(label).long()
        path = self.df.EEG_Path.iloc[index]
        testing = f'dfd'
        path = f"input/{path}"
        # print('reading ', path)
        image = cv2.imread(path, 0)
        # print('reading successfully and size = ', image.shape)

        try:
            image = cv2.resize(image, (37, 75)) / 255
        except:
            pass
        image = np.reshape(image, (75, 37, 1))
       
        if self.transform is not None:
            image = self.transform(image)

        image = np.transpose(image, (2,0,1)).astype(np.float32)

        image = torch.tensor(image).float()

        
        #torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
        #print(list_of_labels)
        return image, label

SyntaxError: invalid syntax (<ipython-input-12-4485113df4aa>, line 20)

In [14]:
a = f'dfd'

SyntaxError: invalid syntax (<ipython-input-14-ea580dfbad66>, line 1)

In [15]:
!which python

/DATA/rahul_1911mt11/anaconda3/bin/python


In [16]:
!python --version

Python 3.8.5
